# 2. CoLLiE 的使用分析

> 2.1 &ensp; 案例：指令微调 MOSS
> 
> 2.2 &ensp; CoLLiE 的 Config 模块
> 
> 2.3 &ensp; CoLLiE 的 data 和 model
> 
> 2.4 &ensp; CoLLiE 的 Evaluator 模块
> 
> 2.5 &ensp; CoLLiE 的 Trainer 模块

## 2.1 &ensp; 案例：指令微调 MOSS

&ensp; &ensp; 在上一节中，我们已经介绍了 CoLLiE 提出的背景、实现的功能 以及 包含的模块。通过上一节的叙述，想必大家已经可以配置好环境、成功地安装 CoLLiE，并且了解了 CoLLiE 的大致结构，遇到一些的功能需求可以找到对应的代码文件，尝试自行了解相关功能是如何实现的了。

&ensp; &ensp; 在这一节中，我们将介绍 **CoLLiE 的使用流程** 以及 **基础模块的使用**，包括 **Config**、**Monitor**、**Evaluator**、**Trainer**。而正所谓实践出真知，在详细介绍上述模块的使用细节与基础原理前，本小节将先通过一个 CoLLiE 的使用案例，告诉大家 CoLLiE 是如何使用、相关代码是如何运行起来的；接着在后续小节中，详细介绍各个部分代码的运行原理、对应模块的使用细节。

&ensp; &ensp; 本教程使用的CoLLiE使用案例是，使用 LOMO 算法（优化算法），零冗余优化器 ZeRO-3（并行策略），全参数微调 MOSS-003（基座模型），使其过拟合一句话（微调任务）：输入 'Collie is a python package for'，输出 'finetuning large language models .'（训练数据）。完整代码文件见 CoLLiE 代码仓库中的 [examples/finetune_moss_for_training.py](https://github.com/OpenLMLab/collie/blob/dev/examples/finetune_moss_for_training.py)，切换到所在路径后，在 jupyter-notebook 通过如下命令即可完成运行。

```sh
!CUDA_VISIBLE_DEVICES=0,1,2,3 torchrun --standalone --nproc_per_node=4 finetune_moss_for_training.py
```

&ensp; &ensp; 这里重点解释一下这行命令的含义。首先，**CUDA_VISIBLE_DEVICES=0,1,2,3**，表示使用当前服务器上的0至3号显卡。接着，**torchrun xxx.py**，表示以分布式的形式启动 xxx.py脚本，即在0至3号显卡上并行运行xxx.py脚本；至于如何并行，则取决于xxx.py脚本内部的定义；至于torchrun，只要python环境中安装pytorch包，即可以运行。剩下的 **--standalone** 和 **--nproc_per_node=4** 都是 torchrun 命令的参数：前者表示在1台服务器上运行；后者表示当前服务器（node）拥有的显卡数量（每个显卡对应一个进程 process）。

&ensp; &ensp; 关于pytorch分布式相关的内容，这里还需要额外补充 **world_size** 和 **rank** 两个概念。world_size 表示 整个分布式程序对应的子进程数量，rank 表示 每个子进程的标识号；对于上述命令对应的程序，由于其在1台服务器的4张显卡上运行，1卡对应1个子进程，所以world_size=4，0至3号显卡上的子进程rank分别是0到3。

&ensp; &ensp; 关于pytorch分布式相关的更多内容，可以参考 [pytorch官方文档](https://pytorch.org/docs/stable/index.html) 中 [关于torchrun的介绍](https://pytorch.org/docs/stable/elastic/run.html)。在接下来的教程中，我们将打开finetune_moss_for_training.py，一点点讲解前文所述的代码模块、并行策略、优化算法是什么，以及 CoLLiE 是如何实现它们的。

## 2.2 &ensp; CoLLiE 的 Config 模块

&ensp; &ensp; 打开 finetune_moss_for_training.py，除了最开始的import部分，第一个用到的CoLLiE模块就是CollieConfig。**CoLLie 的 CollieConfig 是整个 CoLLiE 的核心**，CoLLie 的高度集成化归功于对 CollieConfig 配置文件的集中统一管理。**CoLLiE 几乎所有的组件都受到 CollieConfig 的控制**，包括模型架构（CollieConfig.model_config）、并行策略（CollieConfig.dp_size, CollieConfig.pp_size, CollieConfig.tp_size）、微调方法（CollieConfig.peft_config）等；CollieConfig 涉及的全部配置参数 以及 对应的功能描述 如下表所示。

| 名称 | 描述 |
|:----|:----|
| seed | 随机数种子，整数，默认 42 | 
| dp_size | 数据并行粒度，整数，默认 1，详见 [tutorial-3](https://openlmlab-collie.readthedocs.io/zh_CN/latest/tutorials/collie-tutorial-3-parallel.html) | 
| pp_size | 流水线并行粒度，整数，默认 1，详见 [tutorial-3](https://openlmlab-collie.readthedocs.io/zh_CN/latest/tutorials/collie-tutorial-3-parallel.html) | 
| tp_size | 张量并行粒度，整数，默认 1，详见 [tutorial-3](https://openlmlab-collie.readthedocs.io/zh_CN/latest/tutorials/collie-tutorial-3-parallel.html) | 
| pp_partition_method | 流水线并行切分方式，默认 'parameters'，详见 [tutorial-3](https://openlmlab-collie.readthedocs.io/zh_CN/latest/tutorials/collie-tutorial-3-parallel.html). | 
| train_epochs | 训练 epoch 数量，整数，默认 100 | 
| eval_per_n_steps | 多少 step 一次评测，整数，默认 0 | 
| eval_per_n_epochs | 多少 epoch 一次评测，整数，默认 0 | 
| train_micro_batch_size | 训练 batch 大小，整数，默认 1（流水线并行时，作为 micro_batch 大小，详见 [tutorial-3](https://openlmlab-collie.readthedocs.io/zh_CN/latest/tutorials/collie-tutorial-3-parallel.html)） | 
| gradient_accumulation_steps | 多少 backward 一次 step，整数，默认 1（流水线并行时，决定 train_batch 大小，详见 [tutorial-3](https://openlmlab-collie.readthedocs.io/zh_CN/latest/tutorials/collie-tutorial-3-parallel.html)） | 
| eval_batch_size | 测试 batch 大小，整数，默认 1 | 
| checkpointing | 是否使用 activation checkpointing，默认 True |
| use_flash | 是否使用 flash attention，默认 True | 
| dropout | dropout 大小，浮点数，默认 0.0 | 
| initization_method | 参数初始化方法，可选值 'none'（默认）、'normal'、'kaiming_normal'、'kaiming_uniform'等 | 
| initization_method_params | 指定参数对应初始化方法，字典型，默认 None |
| low_cpu_mem_usage | 是否在模型初始化阶段尝试减少 CPU 占用，默认 True | 
| ds_config | 指定模型 deepspeed 参数，可以是字典型，可以是文件名 如 "ds_config.json" | 
| model_config | 指定模型架构相关的配置项，默认 [transformers.PretrainedConfig()](https://huggingface.co/docs/transformers/v4.30.0/en/main_classes/configuration#transformers.PretrainedConfig) | 
| peft_config | 指定模型参数高效微调方法，默认 [peft.PeftConfig()](https://huggingface.co/docs/peft/main/en/package_reference/config#peft.PeftConfig) | 
| quantization_config | 指定模型量化方法，默认 [transformers.BitsAndBytesConfig()](https://huggingface.co/docs/transformers/main_classes/quantization#transformers.BitsAndBytesConfig) | 

&ensp; &ensp; 由于在 CollieConfig 的所有的配置项中，模型架构相关的配置是较为繁琐但又相对固定的，因此 CollieConfig 提供了与 transformers.PretrainedConfig 类似的 **from_pretrained 方法初始化 CollieConfig**，并对 CollieConfig.model_config 赋值。例如，在 finetune_moss_for_training.py 中，首先明确模型架构为"fnlp/moss-moon-003-sft"，接着使用 CollieConfig.from_pretrained 快速初始化，再定义并行相关参数（需要注意的是，由于是4卡运行，虽然三个并行粒度全部设为1，但数据并行粒度dp_size会自动设为4，详见 [tutorial-3](https://openlmlab-collie.readthedocs.io/zh_CN/latest/tutorials/collie-tutorial-3-parallel.html)），最后定义训练相关参数，如 训练轮数、评测频率、batch大小。

``` python
from collie.config import CollieConfig

pretrained_model = "fnlp/moss-moon-003-sft"  

config = CollieConfig.from_pretrained(pretrained_model, trust_remote_code=True)
config.dp_size = 1  # 4
config.tp_size = 1
config.pp_size = 1
config.train_epochs = 1
config.eval_per_n_steps = 0
config.eval_per_n_epochs = 1 
config.train_micro_batch_size = 2
config.eval_batch_size = 1
```

## 2.3 &ensp; CoLLiE 的 data 和 model

&ensp; &ensp; model模块列表，自定义模型要求

在所有并行策略中，张量并行（tp）和流水线并行（pp）均对模型架构有严格要求，transformers 提供的模型代码并不能满足要求，因此我们在 CoLLie 中重写了一些较为流行的模型，并保证接口和 transformers 尽可能一致。包括：LLAMA、MOSS、ChatGLM、ChatGLM2、InternLM
这些重写后的模型一方面可以使用更加多样性的并行策略，同时我们也对他们加入了 flash attention 支持。来自 transformers 的模型 CoLLie 也是同样支持的，不过并行策略只能使用zero。

&ensp; &ensp; CollieDatasetForTraining对数据的转换

&ensp; &ensp; 在`finetune_moss_for_training.py`脚本中，

``` python
from collie.data import CollieDatasetForTraining
from collie.data import CollieDataLoader

from collie.models.moss_moon import Moss003MoonForCausalLM

train_dataset = [
    {
        'input': 'Collie is a python package for ',
        'output': 'finetuning large language models.'
    } for _ in range(10000)
]
train_dataset = CollieDatasetForTraining(train_dataset, tokenizer)
eval_dataset = train_dataset[:32]

model = Moss003MoonForCausalLM.from_pretrained(pretrained_model, config=config)
``` 

&ensp; &ensp; optim简单介绍

&ensp; &ensp; 跳过 Callback、Monitor，详见 [tutorial-4](https://openlmlab-collie.readthedocs.io/zh_CN/latest/tutorials/collie-tutorial-4-lomo-peft.html)

## 2.4 &ensp; CoLLiE 的 Evaluator 模块

&ensp; &ensp; evaluator模块列表

| 名称 | 描述 |
|:----|:----|
| Evaluator | | 
| EvaluatorForPerplexity | | 
| EvaluatorForClassfication | | 
| EvaluatorForGeneration | | 

&ensp; &ensp; metric模块列表

| 名称 | 描述 |
|:----|:----|
| BaseMetric | |
| DecodeMetric | |
| AccuracyMetric | |
| PPLMetric | |
| BleuMetric | |
| RougeMetric | |
| ClassifyFPreRecMetric | |

&ensp; &ensp; 在`finetune_moss_for_training.py`脚本中，

``` python
from collie.controller.evaluator import EvaluatorForPerplexity, EvaluatorForGeneration

from collie.metrics import PPLMetric, DecodeMetric

evaluator_ppl = EvaluatorForPerplexity(
    model=model, config=config, dataset=eval_dataset,
    monitors=[EvalMonitor(config), ], metrics={'ppl': PPLMetric(), }
)

evaluator_decode = EvaluatorForGeneration(
    model=model, config=config, tokenizer=tokenizer, dataset=eval_dataset,
    monitors=[EvalMonitor(config), ], metrics={'decode': DecodeMetric(), }
)
``` 

## 2.5 &ensp; CoLLiE 的 Trainer 模块

&ensp; &ensp; trainer参数列表

| 名称 | 描述 |
|:----|:----|
| model | torch.nn.Module,, | 
| config | CollieConfig,, | 
| tokenizer | PreTrainedTokenizerBase,None, | 
| loss_fn | Callable,GPTLMLoss(), | 
| train_fn | Callable,None, | 
| eval_fn | Callable,None, | 
| optimizer | torch.optim.Optimizer,None, | 
| lr_scheduler | LRScheduler, DeepSpeedSchedulerCallable,None, | 
| train_dataset | torch.utils.data.Dataset,None,CollieDatasetForTraining | 
| eval_dataset | torch.utils.data.Dataset,None,CollieDatasetForTraining | 
| callbacks | Callback, List[Callback],None, | 
| train_dataset_collate_fn | Callable,ColliePadder(), | 
| eval_dataset_collate_fn | Callable,ColliePadder(padding_left=True), | 
| data_provider | BaseProvider,None, | 
| monitors | BaseMonitor,[], | 
| metrics | Dict,None, | 
| evaluators | List,None, | 

&ensp; &ensp; 在`finetune_moss_for_training.py`脚本中，

``` python
from collie.controller.trainer import Trainer

trainer = Trainer(
    model=model,config=config,train_dataset=train_dataset,
    loss_fn=GPTLMLoss(-100),optimizer=optimizer,
    monitors=monitors, evaluators=[evaluator_ppl, evaluator_decode]
)

trainer.train()
```